<a href="https://colab.research.google.com/github/oscarboudap/gravitationalWaves/blob/main/GW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycbc torch h5py pandas scikit-learn matplotlib tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [ ]:
import numpy as np
import os
import json
import torch
from pycbc.waveform import get_td_waveform
from scipy.signal import spectrogram
from tqdm import tqdm

# Carpeta de salida
output_dir = "gw_dataset"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(f"{output_dir}/signals", exist_ok=True)
os.makedirs(f"{output_dir}/metadata", exist_ok=True)

# Número de muestras a generar
N = 5000
for i in tqdm(range(N), desc="Generando señales"):
    m1 = np.random.uniform(20, 80)
    m2 = np.random.uniform(10, m1)
    spin1z = np.random.uniform(-0.99, 0.99)
    spin2z = np.random.uniform(-0.99, 0.99)
    distance = np.random.uniform(100, 1000)
    redshift = np.random.uniform(0.06, 0.12)       # centrado alrededor de ~0.09
    eccentricity = np.random.uniform(0.0, 0.05)    # la mayoría de GWs tienen órbitas casi circulares
    polarization = np.random.uniform(0.0, 2*np.pi)
    ra = np.random.uniform(0.0, 2*np.pi)
    dec = np.random.uniform(-np.pi/2, np.pi/2)
    tc = np.random.uniform(-0.1, 0.1)

    # Clasificación del tipo de fuente
    if m1 < 3 and m2 < 3:
        source_type = "BNS"
    elif m1 >= 5 and m2 >= 5:
        source_type = "BBH"
    else:
        source_type = "BHNS"

    try:
        hp, hc = get_td_waveform(
            approximant="IMRPhenomD",
            mass1=m1,
            mass2=m2,
            spin1z=spin1z,
            spin2z=spin2z,
            delta_t=1.0/8192,    # 🔧 MÁS RESOLUCIÓN
            f_lower=10,          # 🔧 MÁS BAJA FRECUENCIA
            distance=distance
        )
    except Exception as e:
        print(f"Error en muestra {i}: {e}")
        continue

    noise = np.random.normal(0, 1e-22, len(hp))
    hp += noise

    np.save(f"{output_dir}/signals/hp_{i}.npy", hp.numpy())
    np.save(f"{output_dir}/signals/hc_{i}.npy", hc.numpy())

    metadata = {
        "m1": m1,
        "m2": m2,
        "spin1z": spin1z,
        "spin2z": spin2z,
        "distance": distance,
        "redshift": redshift,
        "eccentricity": eccentricity,
        "polarization": polarization,
        "ra": ra,
        "dec": dec,
        "tc": tc,
        "source_type": source_type,
        "sample_rate": 8192,
        "delta_t": float(hp.delta_t),
        "f_lower": 10,
        "length": len(hp)
    }

    with open(f"{output_dir}/metadata/params_{i}.json", "w") as f:
        json.dump(metadata, f, indent=2)

# 🔧 Normalización y espectrogramas
targets_min = np.array([5, 5, -0.99, -0.99, 100, 0.06, 0.0, 0.0, 0.0, -np.pi/2, -0.1])
targets_max = np.array([80, 80, 0.99, 0.99, 1000, 0.12, 0.05, 2*np.pi, 2*np.pi, np.pi/2, 0.1])

X_list, y_list, class_list = [], [], []
signal_dir = "gw_dataset/signals"
meta_dir = "gw_dataset/metadata"
source_type_map = {"BBH": 0, "BNS": 1, "BHNS": 2}

valid_count = 0
for i in tqdm(range(N), desc="Procesando espectrogramas"):
    try:
        signal_path = os.path.join(signal_dir, f"hp_{i}.npy")
        meta_path = os.path.join(meta_dir, f"params_{i}.json")

        hp = np.load(signal_path)
        with open(meta_path, "r") as f:
            meta = json.load(f)

        f_, t_, Sxx = spectrogram(hp, fs=8192, nperseg=128)
        Sxx = np.log10(Sxx + 1e-10)

        # 🔧 Ajustar tamaño exacto 64x64 con padding si es necesario
        Sxx = Sxx[:64, :64]
        pad_h = 64 - Sxx.shape[0]
        pad_w = 64 - Sxx.shape[1]
        Sxx = np.pad(Sxx, ((0, pad_h), (0, pad_w)), mode='constant', constant_values=0)

        X_list.append(Sxx)
        valid_count += 1

        targets = np.array([
            meta["m1"], meta["m2"], meta["spin1z"], meta["spin2z"],
            meta["distance"], meta["redshift"], meta["eccentricity"],
            meta["polarization"], meta["ra"], meta["dec"], meta["tc"]
        ])
        targets_norm = (targets - targets_min) / (targets_max - targets_min)
        y_list.append(targets_norm)
        class_list.append(source_type_map[meta["source_type"]])

    except Exception as e:
        print(f"⚠️ Error en muestra {i}: {e}")
        continue

print(f"✅ Señales válidas generadas: {valid_count}/{N}")

# Convertir a tensores
X = torch.tensor(np.stack(X_list)).unsqueeze(1).float()
y = torch.tensor(np.stack(y_list)).float()
labels = torch.tensor(class_list).long()


Procesando espectrogramas: 100%|██████████| 5000/5000 [01:55<00:00, 43.12it/s]


✅ Señales válidas generadas: 5000/5000


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SurrogateCNN(nn.Module):
    def __init__(self):
        super(SurrogateCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(128 * 6 * 6, 256)
        self.fc_reg = nn.Linear(256, 11)
        self.fc_cls = nn.Linear(256, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        out_reg = self.fc_reg(x)
        out_cls = F.log_softmax(self.fc_cls(x), dim=1)  # ✅ log_softmax
        return out_reg, out_cls

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

model = SurrogateCNN()
dataset = TensorDataset(X, y, labels)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

optimizer = optim.Adam(model.parameters(), lr=1e-5)
loss_reg = nn.MSELoss()
loss_cls = nn.NLLLoss()

alpha = 1.0
beta = 0.1

prev_loss = None
patience_counter = 0
patience_limit = 5
tolerance = 0.01
max_epochs = 10  # por si tarda mucho

for epoch in range(max_epochs):
    model.train()
    total_loss = 0
    for xb, yb, lb in loader:
        pred_reg, pred_cls = model(xb)
        loss = alpha * loss_reg(pred_reg, yb) + beta * loss_cls(pred_cls, lb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Total Loss: {total_loss:.4f}")

    # Comprobación de convergencia
    if prev_loss is not None and abs(total_loss - prev_loss) < tolerance:
        patience_counter += 1
    else:
        patience_counter = 0

    if patience_counter >= patience_limit:
        print(f"✅ Convergencia alcanzada en la época {epoch+1}. Deteniendo entrenamiento.")
        break

    prev_loss = total_loss

# Guardar modelo final
torch.save(model.state_dict(), "modelo_surrogado_weights.pt")


Epoch 1, Total Loss: 17.7165
Epoch 2, Total Loss: 12.3919
Epoch 3, Total Loss: 12.2778
Epoch 4, Total Loss: 12.2239
Epoch 5, Total Loss: 12.2025
Epoch 6, Total Loss: 12.1911
Epoch 7, Total Loss: 12.2367
Epoch 8, Total Loss: 12.1838
Epoch 9, Total Loss: 12.1921
Epoch 10, Total Loss: 12.1680


In [ ]:
model = SurrogateCNN()
model.load_state_dict(torch.load("modelo_surrogado_weights.pt", map_location="cpu"))
model.eval()

with torch.no_grad():
    pred_reg, pred_cls = model(X[0:1])
    pred_reg = pred_reg.numpy()[0]
    pred_cls = pred_cls.argmax(dim=1).item()

    # Desnormalización
    pred_reg = pred_reg * (targets_max - targets_min) + targets_min

    # Validación física
    pred_reg[6] = np.clip(pred_reg[6], 0, 1)  # excentricidad
    pred_reg[9] = np.clip(pred_reg[9], -np.pi/2, np.pi/2)  # declinación
    pred_reg[10] = np.clip(pred_reg[10], -0.1, 0.1)  # tc

    print("Predicción física:")
    for name, val in zip([
        "m1", "m2", "spin1z", "spin2z", "distance", "redshift",
        "eccentricity", "polarization", "ra", "dec", "tc"
    ], pred_reg):
        print(f"{name}: {val:.4f}")

    tipo_fuente = ["BBH", "BNS", "BHNS"][pred_cls]
    print("Tipo de fuente predicho:", tipo_fuente)


Predicción física:
m1: 48.3469
m2: 29.3488
spin1z: 0.0096
spin2z: 0.0565
distance: 568.7696
redshift: 0.0913
eccentricity: 0.0250
polarization: 3.2191
ra: 3.0796
dec: -0.0095
tc: 0.0005
Tipo de fuente predicho: BBH


In [ ]:
# ID de la muestra sobre la que hiciste la predicción
sample_id = 0

# Cargar valores reales desde el JSON correspondiente
with open(f"gw_dataset/metadata/params_{sample_id}.json") as f:
    valores_reales = json.load(f)

# Emparejar campos relevantes
campos = ["m1", "m2", "spin1z", "spin2z", "distance", "redshift", "eccentricity", "polarization", "ra", "dec", "tc"]
valores_reales_vec = np.array([valores_reales[c] for c in campos])

# Calcular errores
errores_abs = np.abs(pred_reg - valores_reales_vec)
errores_rel = errores_abs / (np.abs(valores_reales_vec) + 1e-8)

# Mostrar resultados
print("\n📊 Comparación modelo vs. real:")
print(f"{'Parámetro':<15} {'Predicho':>10} {'Real':>10} {'Error abs.':>12} {'Error rel.':>12}")
for i, nombre in enumerate(campos):
    print(f"{nombre:<15} {pred_reg[i]:10.4f} {valores_reales_vec[i]:10.4f} {errores_abs[i]:12.4f} {errores_rel[i]*100:11.2f}%")



📊 Comparación modelo vs. real:
Parámetro         Predicho       Real   Error abs.   Error rel.
m1                 48.3469    73.6905      25.3436       34.39%
m2                 29.3488    58.0934      28.7446       49.48%
spin1z              0.0096     0.1126       0.1030       91.47%
spin2z              0.0565    -0.8654       0.9219      106.53%
distance          568.7696   315.5525     253.2171       80.25%
redshift            0.0913     0.0810       0.0103       12.72%
eccentricity        0.0250     0.0022       0.0228     1046.24%
polarization        3.2191     4.3089       1.0897       25.29%
ra                  3.0796     1.1451       1.9346      168.95%
dec                -0.0095     0.8062       0.8157      101.18%
tc                  0.0005     0.0842       0.0837       99.45%
